# This notebook is for preprocessing PTBXL, CPSC2018, and CSN datasets for finetuning tasks.

In [ ]:
import numpy as np
import pandas as pd
import wfdb
import os
import ast
from matplotlib import pyplot as plt
import seaborn as sns
from pprint import pprint
from tqdm import tqdm
from scipy.ndimage import zoom
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [ ]:
# set the split file path to store your processed csv file
split_path = ''
# set the meta path for the raw ecg you download
meta_path = ''

# Preprocessing PTB-XL dataset

In [ ]:
'''
Since PTB-XL provide the offical split, we will use the offical split for the finetune dataset.
The offical preprocess code is shown in the orignal paper: https://www.nature.com/articles/s41597-020-0495-6
We also list the preprocessed csv file in MERL/finetune/data_split/ptbxl
'''

# Preprocessing CPSC2018 Dataset

In [ ]:
'''
This dataset provide raw file in .mat format.
We first convert the .mat file to .hea and .dat file using the wfdb package.
Then we downsample the data to 100Hz and 500Hz.
All information of this dataset can be found in: http://2018.icbeb.org/Challenge.html
'''

# here is your original data folder, you should download the data from the website
ori_data_folder = os.path.join(meta_path, 'icbeb2018')

# here is the output folder to store the preprocessed data
output_folder = os.path.join(meta_path, 'icbeb2018')
output_datafolder_100 = output_folder+ '/records100/'
output_datafolder_500 = output_folder+ '/records500/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
else:
    print('The folder already exists')
if not os.path.exists(output_datafolder_100):
    os.makedirs(output_datafolder_100)
else:
    print('The folder already exists')
if not os.path.exists(output_datafolder_500):
    os.makedirs(output_datafolder_500)
else:
    print('The folder already exists')

# function to store 12 leads ECG data as wfdb format
def store_as_wfdb(signame, data, sigfolder, fs):
    channel_itos=['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    wfdb.wrsamp(signame,
                fs=fs,
                sig_name=channel_itos, 
                p_signal=data,
                units=['mV']*len(channel_itos),
                fmt = ['16']*len(channel_itos), 
                write_dir=sigfolder)  

# load the reference csv file
reference_path = os.path.join(output_folder, 'REFERENCE.csv')
df_reference = pd.read_csv(reference_path)

# define the label dictionary
# label_dict = {1:'NORM', 2:'AFIB', 3:'1AVB', 4:'CLBBB', 5:'CRBBB', 6:'PAC', 7:'VPC', 8:'STD_', 9:'STE_'}
label_dict = {1:'NORM', 2:'AFIB', 3:'1AVB', 4:'CLBBB', 5:'CRBBB', 6:'PAC', 7:'VPC', 8:'STD', 9:'STE'}

data = {'ecg_id':[], 'filename':[], 'validation':[], 'age':[], 'sex':[], 'scp_codes':[]}

# read all .mat files from the folder then convert to .hea and .dat files
ecg_counter = 0
for folder in ['all_data']:
    filenames = os.listdir(os.path.join(ori_data_folder, folder))
    for filename in tqdm(filenames):
        if filename.split('.')[1] == 'mat':
            ecg_counter += 1
            name = filename.split('.')[0]

            sex, age, sig = loadmat(ori_data_folder + '/' + folder + '/' + filename)['ECG'][0][0]
            data['ecg_id'].append(ecg_counter)
            data['filename'].append(name)
            data['validation'].append(False)
            data['age'].append(age[0][0])
            data['sex'].append(1 if sex[0] == 'Male' else 0)
            labels = df_reference[df_reference.Recording == name][['First_label' ,'Second_label' ,'Third_label']].values.flatten()
            labels = labels[~np.isnan(labels)].astype(int)
            data['scp_codes'].append({label_dict[key]:1 for key in labels})

            # # resample to 500 hz data
            # store_as_wfdb(str(ecg_counter), sig.T, output_datafolder_500, 500)
            # # resample to 100 hz data
            # down_sig = np.array([zoom(channel, .2) for channel in sig])
            # store_as_wfdb(str(ecg_counter), down_sig.T, output_datafolder_100, 100)

df = pd.DataFrame(data)
df['patient_id'] = df.ecg_id
# df = stratisfy_df(df, 'strat_fold')
# df.to_csv(output_folder+'icbeb_database.csv')

In [ ]:
# make the patient_id column the first column
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
switched_df = df[cols]

In [ ]:
# Extract all unique labels from the 'scp_codes' column
# all_labels = set()
# for item in switched_df['scp_codes']:
#     all_labels.update(item.keys())

all_labels = ['AFIB', 'VPC', 'NORM', '1AVB', 'CRBBB', 'STE', 'PAC', 'CLBBB', 'STD']


# # Create new columns for each label
for label in all_labels:
    switched_df[label] = switched_df['scp_codes'].apply(lambda x: x.get(label, 0))

cols = list(switched_df.columns)
print(cols)
# cols[-1] = 'STD'
# cols[-4] = 'STE'
# # replace columns name
# switched_df.columns = cols


In [ ]:
# split train test val
train_df, test_df = train_test_split(switched_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

print(f'train_df shape: {train_df.shape}')
print(f'val_df shape: {val_df.shape}')
print(f'test_df shape: {test_df.shape}')

# save the csv files
# train_df.to_csv(split_path+'icbeb_train.csv', index=False)
# val_df.to_csv(split_path+'icbeb_val.csv', index=False)
# test_df.to_csv(split_path+'icbeb_test.csv', index=False)


# Preprocessing CSN Dataset

In [ ]:
'''
For all details of the dataset, please refer to: https://physionet.org/content/ecg-arrhythmia/1.0.0/
'''

your_path = meta_path

data_path = f'{your_path}chapman/WFDBRecords'
folders = os.listdir(data_path)
num_folders = len(folders)
folders = sorted(folders)
folders = [os.path.join(data_path, f) for f in folders]
folders = [f for f in folders if os.path.isdir(f)]

dict_with_empty_lists = {f"{i:02d}": [] for i in range(1, 47)}
for i, folder in enumerate(folders):
    subfolders = os.listdir(folder)
    subfolders = sorted(subfolders)
    subfolders = [os.path.join(folder, f) for f in subfolders]
    subfolders = [f for f in subfolders if os.path.isdir(f)]
    dict_with_empty_lists[f"{i+1:02d}"] = subfolders


# place this '/raid/cl522/ecg-text/downstream' with your own path
for key in dict_with_empty_lists.keys():
    dict_with_empty_lists[key] = [x.replace(f'{your_path}', '') for x in dict_with_empty_lists[key]]

def read_header_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        header_info = [line.strip() for line in lines]
    return header_info

df = {'ecg_path': [], 
      'age': [], 
      'diagnose': []}

ref = pd.read_csv(f'{your_path}chapman/ConditionNames_SNOMED-CT.csv')
ref['Snomed_CT'] = ref['Snomed_CT'].astype(str)

# count the number of mat file in each folder
total_files = 0
for key in tqdm(dict_with_empty_lists.keys()):
    for folder in dict_with_empty_lists[key]:
        files = os.listdir(f'{your_path}'+folder)
        mat_files = [f for f in files if f.endswith('.mat')]
        hea_files = [f for f in files if f.endswith('.hea')]
        
        mat_files_path = [os.path.join(f'{your_path}', folder, f) for f in mat_files]
        hea_files_path = [os.path.join(f'{your_path}', folder, f) for f in hea_files]
        mat_files_path = sorted(mat_files_path)
        hea_files_path = sorted(hea_files_path)

        for file, hea_file in zip(mat_files_path, hea_files_path):
            mat = loadmat(file)
            ecg = mat['val']
            hea = read_header_file(hea_file)
            
            df['ecg_path'].append(file)
            df['age'].append(hea[0].split()[1])
            
            try:
                diagnose_str = []
                Dx_idx = [i for i, s in enumerate(hea) if 'Dx' in s]
                diagnose_code = hea[Dx_idx[0]].split()[1]
                diagnose_code = diagnose_code.split(',')
                for i in range(len(diagnose_code)):
                    diagnose = ref[ref['Snomed_CT'] == diagnose_code[i]]['Acronym Name']
                    diagnose = diagnose.values[0]
                    diagnose_str.append(diagnose)
                diagnose_str = ','.join(diagnose_str)
                df['diagnose'].append(diagnose_str)
            except:
                df['diagnose'].append('Unknown')



In [ ]:
new_df = pd.DataFrame(df)
new_df = new_df[new_df['diagnose'] != 'Unknown']
new_df.reset_index(inplace=True, drop=True)

unique_labels = []
for labels in new_df['diagnose']:
    labels = labels.split(',')
    unique_labels.extend(labels)

unique_labels = list(set(unique_labels))
# Create new columns for each unique label
for label in unique_labels:
    new_df[label] = new_df['diagnose'].apply(lambda x: 1 if label in x else 0)

In [ ]:
# count the number of sample for each label
label_count = {}
for label in unique_labels:
    label_count[label] = new_df[label].sum()
# sort the label_count dictionary
label_count = dict(sorted(label_count.items(), key=lambda item: item[1], reverse=True))
# drop the label with less than 10 samples
for key in list(label_count.keys()):
    if label_count[key] < 10:
        del label_count[key]
# drop the columns not in label_count
for key in list(new_df.columns):
    if key not in label_count.keys():
        new_df.drop(key, axis=1, inplace=True)

In [ ]:
# split train test val
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'train_df shape: {train_df.shape}')
print(f'val_df shape: {val_df.shape}')
print(f'test_df shape: {test_df.shape}')

# save the csv files
# train_df.to_csv(f'{split_path}chapman/'+'chapman_train.csv', index=False)
# val_df.to_csv(f'{split_path}chapman/'+'chapman_val.csv', index=False)
# test_df.to_csv(f'{split_path}chapman/'+'chapman_test.csv', index=False)